In [ ]:
pwd

In [ ]:
cd Rar

In [ ]:
cd object-detection-nn

In [ ]:
# cd ../../..

In [ ]:
remote_mode = True

import os
import numpy as np
from nn.YOLO_VGG16_OBB.utils.constants import ANCHORS
from nn.YOLO_VGG16_OBB.prepare_data.dota_dataset_memory import DotaDataset
from nn.YOLO_VGG16_OBB.prepare_data.transforms import train_transform, test_transform
from nn.YOLO_VGG16_OBB.utils.helpers import convert_cells_to_bboxes, load_checkpoint, nms, plot_image, save_checkpoint
from nn.YOLO_VGG16_OBB.utils.constants import device, s, leanring_rate, save_model, checkpoint_file
from nn.YOLO_VGG16_OBB.model.YOLO_VGG16_OBB import YOLO_VGG16_OBB
import torch
import torch.optim as optim
from nn.YOLO_VGG16_OBB.model.loss import YOLOLoss
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as T

if remote_mode:
    model_path_base = f"/home/dcor/niskhizov/Rar/object-detection-nn/nn/YOLO_VGG16_OBB/notebooks/vgg_f_obb_model"
else:
    model_path_base = f"nn/YOLO_VGG16_OBB/notebooks/"


In [ ]:
categories = ['plane','ship', 'storage-tank', 'baseball-diamond', 'tennis-court', 'basketball-court', 'ground-track-field', 'harbor', 'bridge', 'large-vehicle', 'small-vehicle', 'helicopter', 'roundabout', 'soccer-ball-field', 'swimming-pool']


In [ ]:
# Creating the model from YOLOv3 class 
load_model = True
save_model = False
model = YOLO_VGG16_OBB(num_classes=len(categories)).to(device) 

# Defining the optimizer 
optimizer = optim.Adam(model.parameters(), lr = leanring_rate) 

# Defining the loss function 
loss_fn = YOLOLoss() 

# Defining the scaler for mixed precision training 
scaler = torch.amp.GradScaler(device=device) 
# Loading the checkpoint 
if load_model: 
    load_checkpoint(model_path_base + f"e3300_vgg16_{checkpoint_file}", model, optimizer, leanring_rate, device) 

# Initialize TensorBoard writer
# writer = SummaryWriter(log_dir='runs/YOLO_VGG16_OBB_v2')



In [ ]:
dataset = DotaDataset( 
	categories=categories,
	grid_sizes=[13, 26, 52], 
	anchors=ANCHORS, 
	transform=train_transform 
) 

# Defining the train data loader 
train_loader = torch.utils.data.DataLoader( 
	dataset=dataset, 
	batch_size=8, 
	shuffle=True, 
	num_workers=2,
 	prefetch_factor=10,
) 


In [ ]:
val_dataset = DotaDataset(
    categories=categories,
    grid_sizes=[13, 26, 52],
    anchors=ANCHORS,
    transform=test_transform,  # Use the same transform for validation
    data_base_path = f"nn/dotadataset/train"
)

# Create the validation data loader
val_loader = torch.utils.data.DataLoader(
    dataset=val_dataset,
    batch_size=8,
    shuffle=True,
)

val_loader_iter = iter(val_loader)

In [ ]:
# Scaling the anchors 
scaled_anchors = ( 
	torch.tensor(ANCHORS) *
	torch.tensor(s).unsqueeze(1).unsqueeze(1).repeat(1,3,2) 
).to(device) 

In [ ]:
model.eval()

# Getting a sample image from the test data loader 
try:
	x, y = next(val_loader_iter)
except StopIteration:
	val_loader_iter = iter(val_loader)
	x, y = next(val_loader_iter)
x = x.to(device) 

print("###################################### display and report image ######################################")
with torch.no_grad():
	scaled_anchors = ( 
	torch.tensor(ANCHORS) *
	torch.tensor(s).unsqueeze(1).unsqueeze(1).repeat(1,3,2) 
	).to(device) 
	output = model(x)
	y0, y1, y2 = ( 
		y[0].to(device), 
		y[1].to(device), 
		y[2].to(device), 
	) 

	with torch.amp.autocast(device_type=device): 
		# Getting the model predictions 
		outputs = model(x) 
		# Calculating the loss at each scale 
		loss = ( 
			loss_fn(outputs[0], y0, scaled_anchors[0]) 
			+ loss_fn(outputs[1], y1, scaled_anchors[1]) 
			+ loss_fn(outputs[2], y2, scaled_anchors[2]) 
		) 

	# TEMP- print target boxes
	bboxes = [[] for _ in range(x.shape[0])]
	for i in range(3):
		batch_size, A, S, _, _ = y[i].shape
		anchor = scaled_anchors[i]
		boxes_scale_i = convert_cells_to_bboxes(output[i], anchor, s=S, is_predictions=True)
		for idx, box in enumerate(boxes_scale_i):
			bboxes[idx] += box

	i = 0
	print('bboxes[i] shape:', np.array(bboxes[i]).shape)
	nms_boxes = nms(bboxes[i], iou_threshold=0.6, threshold=0.8)
	img_with_boxes = plot_image(x[i].permute(1, 2, 0).detach().cpu(), nms_boxes, categories)
	img_with_boxes = T.ToTensor()(img_with_boxes)

	# # Print predictions
	# writer.add_scalar('Loss/val', loss.item(), e * len(train_loader) + batch_idx)

	# bboxes = [[] for _ in range(x.shape[0])]
	# for i in range(3):
	# 	batch_size, A, S, _, _ = output[i].shape
	# 	anchor = scaled_anchors[i]
	# 	boxes_scale_i = convert_cells_to_bboxes(output[i], anchor, s=S, is_predictions=True)
	# 	for idx, box in enumerate(boxes_scale_i):
	# 		bboxes[idx] += box

	# i = 0
	# print('bboxes[i] shape:', np.array(bboxes[i]).shape)
	# nms_boxes = nms(bboxes[i], iou_threshold=0.5, threshold=0.6)
	# img_with_boxes = plot_image(x[i].permute(1, 2, 0).detach().cpu(), nms_boxes, categories)
	# img_with_boxes = T.ToTensor()(img_with_boxes)
	# writer.add_image(f'Val/Image_{e}_{i}_{batch_idx}_before', img_with_boxes, e * len(train_loader) + batch_idx)

# model.train()
# except Exception as error:
# 	print(error)
# 	error_counter += 1
# 	if error_counter > 10:
# 		raise error


#################
# training_loop(train_loader, model, optimizer, loss_fn, scaler, scaled_anchors) 


In [ ]:
import cv2
import torch
import matplotlib.pyplot as plt
import numpy as np
from nn.YOLO_VGG16_OBB.utils.constants import device
# Defining a function to calculate Intersection over Union (IoU)


def iou(box1, box2, is_pred=True):
    if is_pred:
        # IoU score for prediction and label
        # box1 (prediction) and box2 (label) are both in [x, y, width, height, angle] format
        # Convert boxes to polygons
        polys1 = []
        polys2 = []

        # angle (box1[..., 4]) is in radian than- convert to degree
        angle1 = box1[..., 4] * (torch.pi / 2)
        angle2 = box2[..., 4] * (torch.pi / 2)
        angle1_degree = torch.rad2deg(angle1)
        angle2_degree = torch.rad2deg(angle2)

        for i in range(box1.shape[0]):
            poly1 = cv2.boxPoints(((box1[i, 0].item(), box1[i, 1].item(
            )), (box1[i, 2].item(), box1[i, 3].item()), angle1_degree[i].item()))
            poly2 = cv2.boxPoints(((box2[i, 0].item(), box2[i, 1].item(
            )), (box2[i, 2].item(), box2[i, 3].item()), angle2_degree[i].item()))
            polys1.append(poly1)
            polys2.append(poly2)

        # Convert polygons to torch tensors
        poly1 = np.array(polys1, dtype=np.float32)
        poly2 = np.array(polys2, dtype=np.float32)
        # Calculate intersection area
        inter_area = polygon_intersection_area(poly1, poly2)
        # Calculate union area
        box1_area = box1[..., 2] * box1[..., 3]
        box2_area = box2[..., 2] * box2[..., 3]
        union_area = box1_area + box2_area - inter_area

        # Calculate IoU score
        epsilon = 1e-6
        iou_score = inter_area / (union_area + epsilon)

        # Return IoU score
        return iou_score.unsqueeze(1)

    else:
        # IoU score based on width and height of bounding boxes

        # Calculate intersection area
        intersection_area = torch.min(
            box1[..., 0], box2[..., 0]) * torch.min(box1[..., 1], box2[..., 1])

        # Calculate union area
        box1_area = box1[..., 0] * box1[..., 1]
        box2_area = box2[..., 0] * box2[..., 1]
        union_area = box1_area + box2_area - intersection_area

        # Calculate IoU score
        iou_score = intersection_area / union_area

        # Return IoU score
        return iou_score.unsqueeze(1)



def polygon_intersection_area(poly1_np, poly2_np):
    # Ensure tensors are on CPU and convert to NumPy
    # poly1_np = poly1.detach().cpu().numpy().astype(np.float32)
    # poly2_np = poly2.detach().cpu().numpy().astype(np.float32)

    inter_areas = []
    for p1, p2 in zip(poly1_np, poly2_np):
        inter_poly = cv2.intersectConvexConvex(p1, p2)
        if inter_poly[0] > 0 and inter_poly[1] is not None:
            inter_areas.append(cv2.contourArea(inter_poly[1]))
        else:
            inter_areas.append(0.0)
    inter_areas = np.array(inter_areas, dtype=np.float32)
    # Convert result back to a tensor
    return torch.tensor(inter_areas, dtype=torch.float32, device=device)


def nms(bboxes_orig, iou_threshold, threshold):
    # Filter out bounding boxes with confidence below the threshold.
    bboxes = [box for box in bboxes_orig if box[1] > threshold]

    # Sort the bounding boxes by confidence in descending order.
    bboxes = sorted(bboxes, key=lambda x: x[1], reverse=True)

    # Initialize the list of bounding boxes after non-maximum suppression.
    if (len(bboxes) > 0):
        first_box = bboxes.pop(0)
        bboxes_nms = [first_box]
    else:
        bboxes_nms = [max(bboxes_orig, key=lambda x: x[1])]

    while len(bboxes) >= 0:
        # Iterate over the remaining bounding boxes.
        for box in bboxes:
            # If the bounding boxes do not overlap or if the first bounding box has
            # a higher confidence, then add the second bounding box to the list of
            # bounding boxes after non-maximum suppression.
            if box[0] != first_box[0] or iou(
                    torch.tensor([first_box[2:]], device=device),
                    torch.tensor([box[2:]], device=device),
            ) < iou_threshold:
                # Check if box is not in bboxes_nms
                if box not in bboxes_nms:
                    # Add box to bboxes_nms
                    bboxes_nms.append(box)

        # Get the first bounding box.
        if len(bboxes) > 0:
            first_box = bboxes.pop(0)
        else:
            break

    # Return bounding boxes after non-maximum suppression.
    # box concist: [class_pred, score, x, y, width, height, angle]
    return bboxes_nms

In [ ]:
for box in bboxes[0]: 
    print(box[1])
    if box[1] > 0.03: 
        print(box)

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from nn.YOLO_VGG16_OBB.utils.helpers import iou

def plot_image(image, boxes, labels, display=True):
    # Getting the color map from matplotlib
    colour_map = plt.get_cmap("tab20b")

    # Convert image to NumPy array (if not already)
    img = np.array(image)
    h, w, _ = img.shape

    # Copy the image to avoid modifying the original
    img_drawn = img.copy()

    # Plot bounding boxes and labels
    for box in boxes:
        class_pred = int(box[0])
        cx, cy, bw, bh, angle = box[2:]

        # Convert to absolute coordinates
        cx, cy, bw, bh = cx * w, cy * h, bw * w, bh * h

        # Get color
        color = tuple(int(c * 255) for c in colour_map(class_pred + 100)[:3])
        if angle > 10:
            print(angle)
        # Get rotated rectangle
        rect = ((cx, cy), (bw, bh), angle)  # OpenCV expects angle in degrees
        box_points = cv2.boxPoints(rect)  # Get corner points
        box_points = np.int32(box_points)  # Convert to integer

        # Draw the rotated rectangle
        cv2.polylines(img_drawn, [box_points], isClosed=True, color=color, thickness=1)

        # Put label text near the rectangle
        # label = labels[class_pred]
        # (text_width, text_height), baseline = cv2.getTextSize(
        #     label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
        # text_x, text_y = int(cx - text_width / 2), int(cy - bh / 2 - 10)
        # cv2.rectangle(img_drawn, (text_x, text_y - text_height - 4),
        #               (text_x + text_width, text_y), color, -1)
        # cv2.putText(img_drawn, label, (text_x, text_y),
        #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    if display:
        # Display the image
        plt.figure(figsize=(8, 6))
        plt.imshow(cv2.cvtColor(img_drawn, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB
        plt.axis("off")
        plt.show()

    return img_drawn  # Return the modified image with drawn bounding boxes


In [ ]:
def nms(bboxes_orig, iou_threshold, threshold):
    # Filter out bounding boxes with confidence below the threshold.
    bboxes = [box for box in bboxes_orig if box[1] > threshold]

    # Sort the bounding boxes by confidence in descending order.
    bboxes = sorted(bboxes, key=lambda x: x[1], reverse=True)

    # Initialize the list of bounding boxes after non-maximum suppression.
    if (len(bboxes) > 0):
        first_box = bboxes.pop(0)
        bboxes_nms = [first_box]
    else:
        bboxes_nms = [max(bboxes_orig, key=lambda x: x[1])]

    while len(bboxes) >= 0:
        # Iterate over the remaining bounding boxes.
        for box in bboxes:
            # If the bounding boxes do not overlap or if the first bounding box has
            # a higher confidence, then add the second bounding box to the list of
            # bounding boxes after non-maximum suppression.
            if box[0] != first_box[0] or iou(
                    torch.tensor([first_box[2:]]),
                    torch.tensor([box[2:]]),
            ) < iou_threshold:
                # Check if box is not in bboxes_nms
                if box not in bboxes_nms:
                    # Add box to bboxes_nms
                    bboxes_nms.append(box)

        # Get the first bounding box.
        if len(bboxes) > 0:
            first_box = bboxes.pop(0)
        else:
            break

    # Return bounding boxes after non-maximum suppression.
    return bboxes_nms

In [ ]:
obj = target[..., 0] == 1
no_obj = target[..., 0] == 0

# Calculating No object loss 
no_object_loss = 4 * bce( 
    (pred[..., 0:1][no_obj]), (target[..., 0:1][no_obj]), 
) 


# Reshaping anchors to match predictions 
anchors = anchors.reshape(1, 3, 1, 1, 2) 
# Box prediction confidence 
box_preds = torch.cat([sigmoid(pred[..., 1:3]), 
                    torch.exp(pred[..., 3:5]) * anchors, pred[..., 5].unsqueeze(-1)
                    ],dim=-1) 
# Calculating intersection over union for prediction and target 
ious = iou(box_preds[obj], target[..., 1:6][obj]).detach() 
ious = ious.unsqueeze(1)
object_loss = mse(sigmoid(pred[..., 0:1][obj]), 
                    ious * target[..., 0:1][obj]) 



In [ ]:
ious.unsqueeze(1).shape

In [ ]:
box_preds[obj].shape

In [ ]:
converted_bboxes = torch.cat(
    (best_class, scores, x, y, width_height, angle.unsqueeze(0)), dim=-1
).reshape(batch_size, num_anchors * s * s, 7)

In [ ]:
angle.shape

In [ ]:
scaled_anchors.shape

In [ ]:
anchor.shape
anchors = anchor.reshape(1, len(anchor), 1, 1, 2)


In [ ]:
anchors.shape

In [ ]:
box_predictions.shape

In [ ]:
box_predictions[..., 2:5].shape

In [ ]:
box_predictions = output[i][..., 1:6]


In [ ]:
torch.exp(box_predictions[..., 2:]) * anchors

In [ ]:
torch.exp(output[i][..., 2:]).shape

In [ ]:
import cv2

In [ ]:
pwd

In [ ]:
cd nn/YOLO_VGG16/prepare_data/

In [ ]:
img_path = f'../../cocodataset/images/train2017/000000111341.jpg'
img = cv2.imread(img_path)


In [ ]:
# iterator = next(iter(train_loader))

In [ ]:
import numpy as np
from nn.YOLO_VGG16_OBB.utils.helpers import iou

In [ ]:
np.array(bboxes[i]).shape

In [ ]:
bboxes_orig, iou_threshold, threshold = bboxes[i], 0.5, 0.6
# Filter out bounding boxes with confidence below the threshold.
bboxes = [box for box in bboxes_orig if box[1] > threshold]

# Sort the bounding boxes by confidence in descending order.
bboxes = sorted(bboxes, key=lambda x: x[1], reverse=True)

# Initialize the list of bounding boxes after non-maximum suppression.
if (len(bboxes) > 0):
    first_box = bboxes.pop(0)
    bboxes_nms = [first_box]
else:
    bboxes_nms = [max(bboxes_orig, key=lambda x: x[1])]

while len(bboxes) >= 0:
    # Iterate over the remaining bounding boxes.
    for box in bboxes:
        # If the bounding boxes do not overlap or if the first bounding box has
        # a higher confidence, then add the second bounding box to the list of
        # bounding boxes after non-maximum suppression.
        if box[0] != first_box[0] or iou(
                torch.tensor(first_box[1:]),
                torch.tensor(box[1:]),
        ) < iou_threshold:
            print('**************************')


In [ ]:
i

In [ ]:
bboxes[5]

In [ ]:
first_box